## Demonstrates some common TensorFlow errors

This notebook demonstrates some common TensorFlow errors, how to find them, and how to fix them.

In [1]:
import tensorflow as tf
print tf.__version__

1.4.1


# Shape error

In [2]:
def some_method(data):
  a = data[:,0:2]
  c = data[:,1]
  s = (a + c)
  return tf.sqrt(tf.matmul(s, tf.transpose(s)))

with tf.Session() as sess:
  fake_data = tf.constant([
      [5.0, 3.0, 7.1],
      [2.3, 4.1, 4.8],
      [2.8, 4.2, 5.6],
      [2.9, 8.3, 7.3]
    ])
  print sess.run(some_method(fake_data))

ValueError: Dimensions must be equal, but are 2 and 4 for 'add' (op: 'Add') with input shapes: [4,2], [4].

In [3]:
def some_method(data):
  a = data[:,0:2]
  print a.get_shape()
  c = data[:,1]
  print c.get_shape()
  s = (a + c)
  return tf.sqrt(tf.matmul(s, tf.transpose(s)))

with tf.Session() as sess:
  fake_data = tf.constant([
      [5.0, 3.0, 7.1],
      [2.3, 4.1, 4.8],
      [2.8, 4.2, 5.6],
      [2.9, 8.3, 7.3]
    ])
  print sess.run(some_method(fake_data))

(4, 2)
(4,)


ValueError: Dimensions must be equal, but are 2 and 4 for 'add_1' (op: 'Add') with input shapes: [4,2], [4].

In [4]:
def some_method(data):
  a = data[:,0:2]
  print a.get_shape()
  c = data[:,1:3]
  print c.get_shape()
  s = (a + c)
  return tf.sqrt(tf.matmul(s, tf.transpose(s)))

with tf.Session() as sess:
  fake_data = tf.constant([
      [5.0, 3.0, 7.1],
      [2.3, 4.1, 4.8],
      [2.8, 4.2, 5.6],
      [2.9, 8.3, 7.3]
    ])
  print sess.run(some_method(fake_data))

(4, 2)
(4, 2)
[[12.884487 11.878131 12.449096 15.721323]
 [11.878131 10.962208 11.489996 14.509306]
 [12.449096 11.489996 12.043255 15.207892]
 [15.721323 14.509306 15.207892 19.204166]]


In [5]:
x = tf.constant([[3, 2],
                 [4, 5],
                 [6, 7]])
print "x.shape", x.shape
expanded = tf.expand_dims(x, 1)
print "expanded.shape", expanded.shape
sliced = tf.slice(x, [0, 1], [2, 1])
print "sliced.shape", sliced.shape

with tf.Session() as sess:
  print "expanded: ", expanded.eval()
  print "sliced: ", sliced.eval()

x.shape (3, 2)
expanded.shape (3, 1, 2)
sliced.shape (2, 1)
expanded:  [[[3 2]]

 [[4 5]]

 [[6 7]]]
sliced:  [[2]
 [5]]


# Vector vs scalar

In [6]:
def some_method(data):
  print data.get_shape()
  a = data[:,0:2]
  print a.get_shape()
  c = data[:,1:3]
  print c.get_shape()
  s = (a + c)
  return tf.sqrt(tf.matmul(s, tf.transpose(s)))

with tf.Session() as sess:
  fake_data = tf.constant([5.0, 3.0, 7.1])
  print sess.run(some_method(fake_data))

(3,)


ValueError: Index out of range using input dim 1; input has only 1 dims for 'strided_slice_6' (op: 'StridedSlice') with input shapes: [3], [2], [2], [2] and with computed input tensors: input[3] = <1 1>.

In [7]:
def some_method(data):
  print data.get_shape()
  a = data[:,0:2]
  print a.get_shape()
  c = data[:,1:3]
  print c.get_shape()
  s = (a + c)
  return tf.sqrt(tf.matmul(s, tf.transpose(s)))

with tf.Session() as sess:
  fake_data = tf.constant([5.0, 3.0, 7.1])
  fake_data = tf.expand_dims(fake_data, 0)
  print sess.run(some_method(fake_data))

(1, 3)
(1, 2)
(1, 2)
[[12.884487]]


# Type error

In [8]:
def some_method(a, b):
  s = (a + b)
  return tf.sqrt(tf.matmul(s, tf.transpose(s)))

with tf.Session() as sess:
  fake_a = tf.constant([
      [5.0, 3.0, 7.1],
      [2.3, 4.1, 4.8],
    ])
  fake_b = tf.constant([
      [2, 4, 5],
      [2, 8, 7]
    ])
  print sess.run(some_method(fake_a, fake_b))

ValueError: Tensor conversion requested dtype float32 for Tensor with dtype int32: 'Tensor("Const_7:0", shape=(2, 3), dtype=int32)'

In [9]:
def some_method(a, b):
  b = tf.cast(b, tf.float32)
  s = (a + b)
  return tf.sqrt(tf.matmul(s, tf.transpose(s)))

with tf.Session() as sess:
  fake_a = tf.constant([
      [5.0, 3.0, 7.1],
      [2.3, 4.1, 4.8],
    ])
  fake_b = tf.constant([
      [2, 4, 5],
      [2, 8, 7]
    ])
  print sess.run(some_method(fake_a, fake_b))

[[15.633618 16.0493  ]
 [16.0493   17.43961 ]]


# TensorFlow debugger

Unfortunately, this method will not work within Datalab. You need to run it in a Terminal window.

In [10]:
%writefile debugger.py
import tensorflow as tf
from tensorflow.python import debug as tf_debug

def some_method(a, b):
  b = tf.cast(b, tf.float32)
  s = (a / b)
  return tf.sqrt(tf.matmul(s, tf.transpose(s)))

with tf.Session() as sess:
  fake_a = tf.constant([
      [5.0, 3.0, 7.1],
      [2.3, 4.1, 4.8],
    ])
  fake_b = tf.constant([
      [2, 0, 5],
      [2, 8, 7]
    ])
    
  sess = tf_debug.LocalCLIDebugWrapperSession(sess)
  sess.add_tensor_filter("has_inf_or_nan", tf_debug.has_inf_or_nan)
  print sess.run(some_method(fake_a, fake_b))

Writing debugger.py


In [11]:
%bash
# Note: doesn't work because Datalab is not an interactive terminal
# You have to ssh into Docker image and run it in a terminal or have TensorFlow locally installed.
# Sorry :(
# python debugger.py --debug

## tf.Print()

In [12]:
%writefile debugger.py
import tensorflow as tf

def some_method(a, b):
  b = tf.cast(b, tf.float32)
  s = (a / b)
  print_ab = tf.Print(s, [a, b])
  s = tf.where(tf.is_nan(s), print_ab, s)
  return tf.sqrt(tf.matmul(s, tf.transpose(s)))

with tf.Session() as sess:
  fake_a = tf.constant([
      [5.0, 3.0, 7.1],
      [2.3, 4.1, 4.8],
    ])
  fake_b = tf.constant([
      [2, 0, 5],
      [2, 8, 7]
    ])
  
  print sess.run(some_method(fake_a, fake_b))

Overwriting debugger.py


In [13]:
!python debugger.py

2018-02-02 17:13:03.217063: I tensorflow/core/platform/cpu_feature_guard.cc:137] Your CPU supports instructions that this TensorFlow binary was not compiled to use: SSE4.1 SSE4.2 AVX
2018-02-02 17:13:03.251004: I tensorflow/core/kernels/logging_ops.cc:79] [[5 3 7.1]...][[2 0 5]...]
[[      nan       nan]
 [      nan 1.4336526]]
